<a href="https://colab.research.google.com/github/axiom19/Resume-Keyword-Assistant/blob/main/ResumeAssistant_Inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Resume Tailor Machine

Motivation: Applying to job makes you use Jobscan, which tells us the missing keywords. Sometimes its hard to incorporate keywords without changing the sense of the statement.
So the following project assists in adding the keyword to your resume.

This notebook implements a Resume Tailor Machine using Gradio and the Hugging Face Gemma 2B model. The application allows users to upload a resume and input a keyword. The AI then incorporates the keyword into the resume in a natural way, improving its relevance for specific job applications.

In [14]:
# ! pip install gradio transformers langchain langchain-huggingface pypdf

In [15]:
# pip install -U langchain-community

## Import Libraries

In [5]:
#import statements

import os
import io
import re
import gradio as gr
import torch
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFaceHub
from langchain.document_loaders import PyPDFLoader

In [7]:
# declare global variables

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

Using device: cpu


## Model Initialization

Here we initialize the Hugging Face model with specific parameters. We're using the google/gemma-2b-it model for this application.

In [8]:
# initialize LLM

model_params = {
    'temperature': 0.2,
    'max_length': 512,
    'repetition_penalty': 1.2,
    'top_p': 0.9
}

llm = HuggingFaceHub(
    repo_id="google/gemma-2-2b-it",
    model_kwargs=model_params,
    huggingfacehub_api_token=os.environ['HF_TOKEN']
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEndpoint`.
  warn_deprecated(


## Prompt Engineering

This section defines the prompt template used to guide the AI in modifying the resume. The prompt instructs the AI to incorporate the given keyword naturally into the resume content.

In [9]:
# create model template

template = """
You are an AI assistant specialized in improving resumes. Given the following resume content and a single keyword, add this keyword naturally into the resume. If the keyword is already present, emphasize it or use it more effectively.

Resume Content:
{resume_content}

Keyword to incorporate:
{keyword}

Add or modify the keyword in the resume and return the following:
1. The section name where the change was made
2. The full line where the keyword was added or modified
3. A one liner explanation of why this change improves the resume

Format your response as follows:
Section: [section name]
Modified Line: [full modified line]
Explanation: [one liner explanation]

If the keyword cannot be appropriately added to the resume, explain why.
"""

# create prompt template
prompt = PromptTemplate(
    template=template,
    input_variables=["resume_content", "keyword"]
)

# create langchain

llm_chain = LLMChain(
    llm=llm,
    prompt=prompt,
)

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(


## Data Processing


In [10]:
# extract data from resume

def extract_data(file_path):
    loader = PyPDFLoader(file_path)
    document = loader.load()

    return document[0].page_content

In [11]:
# resume processing function

def process_resume(file_path, keyword):
    # load the resume
    resume = extract_data(file_path)

    # process the resume
    response = llm_chain.run(resume_content=resume, keyword=keyword)


    section = re.search(r'Section: (.+)', response)
    modified_line = re.search(r'Modified Line: (.+)', response)
    explanation = re.search(r'Explanation: (.+)', response)

    # Format the output
    output = f"Keyword: {keyword}\n\n"
    if section and modified_line and explanation:
        output += f"Section: {section.group(1)}\n"
        output += f"Modified Line: {modified_line.group(1)}\n"
        output += f"Explanation: {explanation.group(1)}\n"
    else:
        output += "The keyword could not be appropriately added to the resume.\n"
        output += f"Explanation: {response}\n"

    return output

In [24]:
# process_resume(file_path="/content/resume.pdf", keyword="innovative")

## Interface Creation

In [22]:
# create UI with Gradio

inputs = [
    gr.File(label="Upload Resume"),
    gr.Textbox(label="Keyword")
]

outputs = gr.Textbox(label="Output")

iface = gr.Interface(
    inputs=inputs,
    outputs=outputs,
    fn=process_resume,
    title="Resume Tailor Machine",
    description="Upload a resume and a keyword to add the keyword and improve the match-score",
)

In [23]:
iface.launch()

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://3bc40299170457d023.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
